In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
# fetch_data_from = current_date - timedelta(days=7)  # Only fetch last 7 days
#! pip install hopsworks==4.1.5 hopsworks-aiomysql==0.2.1

In [5]:
# import sys
# import subprocess

# # Upgrade related dependencies
# subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", "hsfs", "hopsworks", "protobuf"])

In [6]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from ),
    end_time=(fetch_data_to),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-05 01:21:54,134 INFO: Initializing external client
2025-03-05 01:21:54,135 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 01:21:54,754 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641
Fetching data from 2025-02-04 06:21:54.133971+00:00 to 2025-03-05 05:21:54.133971+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.27s) 


In [7]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-04 07:00:00+00:00,2,0
1,2025-02-04 08:00:00+00:00,2,0
2,2025-02-04 09:00:00+00:00,2,0
3,2025-02-04 10:00:00+00:00,2,0
4,2025-02-04 11:00:00+00:00,2,0
...,...,...,...
174440,2025-03-05 01:00:00+00:00,263,10
174441,2025-03-05 02:00:00+00:00,263,5
174442,2025-03-05 03:00:00+00:00,263,3
174443,2025-03-05 04:00:00+00:00,263,4


In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174445 entries, 0 to 174444
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 2.7 MB


In [9]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [10]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174445 entries, 0 to 174444
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 2.7 MB


In [11]:
ts_data = ts_data.rename(columns={"value": "rides"})

In [12]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,48,92,76,8,155,62,60,6,80,...,61,14,13,5,120,218,146,111,263,2025-03-02 00:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,210,2025-02-07 14:00:00
2,3,0,0,0,0,2,0,2,0,1,...,1,1,3,0,2,1,2,1,216,2025-02-04 22:00:00
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,147,2025-02-05 02:00:00
4,2,3,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,217,2025-02-04 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,4,0,1,0,...,0,0,0,1,0,0,0,1,86,2025-02-04 07:00:00
247,2,0,1,1,0,0,2,0,0,2,...,0,3,1,0,1,0,1,0,82,2025-02-13 14:00:00
248,243,130,148,572,217,231,173,141,160,167,...,140,10,0,133,311,225,168,179,142,2025-02-17 00:00:00
249,6,5,22,20,5,23,2,6,1,14,...,2,15,1,0,4,0,24,4,74,2025-03-01 05:00:00


In [14]:
# Ensure `pickup_hour` is timezone-aware in UTC
ts_data["pickup_hour"] = pd.to_datetime(ts_data["pickup_hour"], utc=True)

# Ensure `fetch_data_from` and `fetch_data_to` are also timezone-aware
fetch_data_from = fetch_data_from.tz_convert("UTC") if fetch_data_from.tz else fetch_data_from.tz_localize("UTC")
fetch_data_to = fetch_data_to.tz_convert("UTC") if fetch_data_to.tz else fetch_data_to.tz_localize("UTC")

# Apply filtering
filtered_ts_data = ts_data[ts_data["pickup_hour"].between(fetch_data_from, fetch_data_to)]

print("Filtered ts_data:")
print(filtered_ts_data.head())

Filtered ts_data:
                pickup_hour  pickup_location_id  rides
0 2025-02-08 07:00:00+00:00                 263     32
1 2025-02-20 21:00:00+00:00                 210      0
2 2025-03-01 15:00:00+00:00                 216      3
3 2025-02-13 01:00:00+00:00                 147      0
4 2025-02-20 16:00:00+00:00                 217      2


In [15]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-05 01:22:08,753 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 01:22:08,756 INFO: Initializing external client
2025-03-05 01:22:08,756 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 01:22:09,377 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641
Fetching data from 2025-02-04 06:22:08.753774+00:00 to 2025-03-05 05:22:08.753774+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.11s) 


In [16]:
current_date

Timestamp('2025-03-05 06:22:08.753774+0000', tz='Etc/UTC')

In [17]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-04 07:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-04 07:00:00
2,1,11,7,3,2,3,2,1,7,5,...,1,4,4,3,0,0,1,5,4,2025-03-04 07:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-03-04 07:00:00
4,2,1,2,3,2,3,1,2,4,1,...,0,0,1,0,0,0,0,5,7,2025-03-04 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,259,2025-03-04 07:00:00
247,4,2,0,1,1,1,0,1,1,0,...,1,0,1,0,0,0,1,1,260,2025-03-04 07:00:00
248,4,5,16,21,34,25,31,37,28,33,...,10,1,0,0,1,1,1,1,261,2025-03-04 07:00:00
249,126,177,156,102,81,88,81,79,97,95,...,9,2,2,1,4,4,16,87,262,2025-03-04 07:00:00


In [18]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-05 01:22:19,415 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 01:22:19,417 INFO: Initializing external client
2025-03-05 01:22:19,418 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 01:22:19,960 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215641


In [19]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [20]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,1.0
3,6,0.0
4,7,2.0
...,...,...
246,259,0.0
247,260,1.0
248,261,13.0
249,262,81.0


In [21]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 237, 161, 142, 132,  48, 238, 162, 163, 170], dtype=int32)